In [16]:
import requests
import json
import pandas as pd
from scipy import stats


In [22]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=VOO&outputsize=full&apikey=2XZJV1NODSI2NNNW&'
resp = requests.get(url)
resp.json()['Meta Data']

{'1. Information': 'Daily Time Series with Splits and Dividend Events',
 '2. Symbol': 'VOO',
 '3. Last Refreshed': '2019-09-11 15:13:22',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [17]:
df_trump_2018 = pd.read_json('condensed_2018.json')
df_trump_2017 = pd.read_json('condensed_2017.json')
frames = [df_trump_2018, df_trump_2017]
df_trump = pd.concat(frames)

In [18]:
df_trump_count = df_trump
df_trump_count['date'] = df_trump['created_at'].dt.date
df_trump_count = df_trump_count.groupby(by='date').count()

In [19]:
df_trump_count['count']= df_trump_count['created_at']
df_trump_count['avg_favorites_per_day'] = df_trump.groupby(by='date').sum()['favorite_count']
df_trump_count['avg_favorites_per_day'] = df_trump_count['avg_favorites_per_day']/df_trump_count['count']
df_trump_count = df_trump_count[['count','avg_favorites_per_day']]

In [20]:
df_trump_approval = pd.read_csv('approval_topline.csv')
df_trump_approval = df_trump_approval.loc[df_trump_approval['subgroup'] == 'Voters']
df_trump_approval = df_trump_approval[['approve_estimate', 'disapprove_estimate', 'timestamp']]
df_trump_approval.timestamp = pd.to_datetime(df_trump_approval.timestamp)
df_trump_approval.timestamp = df_trump_approval.timestamp.dt.date

In [109]:


df = pd.DataFrame(resp.json()['Time Series (Daily)'])
pd.to_datetime(df.columns)
df_transposed = df.transpose().set_index(df.columns)
df_filtered = df_transposed.iloc[175:677]
a = list(df_transposed.index)
df_clean = df_filtered.drop(['8. split coefficient','7. dividend amount','6. volume'], axis=1)
df_stocks = df_clean
df_stocks.index = pd.to_datetime(df_stocks.index)
df_stocks.head()


,1. open,2. high,3. low,4. close,5. adjusted close
2018-12-31,229.3100,229.9700,227.4500,229.8100,227.3423
2018-12-28,229.3200,231.0700,226.5200,227.7600,225.3143
2018-12-27,222.9600,228.1200,219.6000,228.0500,225.6012
2018-12-26,216.8800,226.1000,214.8300,225.9500,223.5237
2018-12-24,219.7400,220.8000,215.0700,215.0700,212.7606


In [138]:
df_stocks['1. open'] = df_stocks['1. open'].astype(float)
df_stocks['2. high'] = df_stocks['2. high'].astype(float)
df_stocks['3. low'] = df_stocks['3. low'].astype(float)
df_stocks['4. close'] = df_stocks['4. close'].astype(float)
df_stocks['5. adjusted close'] = df_stocks['5. adjusted close'].astype(float)
df_stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 502 entries, 2018-12-31 to 2017-01-03
Data columns (total 5 columns):
1. open              502 non-null float64
2. high              502 non-null float64
3. low               502 non-null float64
4. close             502 non-null float64
5. adjusted close    502 non-null float64
dtypes: float64(5)
memory usage: 23.5 KB


In [207]:
mean_tweets = df_trump_count['count'].mean()
print(mean_tweets)
a = df_trump_count[df_trump_count['count']<15]
b = df_trump_count[df_trump_count['count']>=15]
a.index

8.54050279329609


Index([2017-01-01, 2017-01-02, 2017-01-03, 2017-01-04, 2017-01-05, 2017-01-06,
       2017-01-07, 2017-01-08, 2017-01-09, 2017-01-10,
       ...
       2018-12-19, 2018-12-22, 2018-12-23, 2018-12-25, 2018-12-26, 2018-12-27,
       2018-12-28, 2018-12-29, 2018-12-30, 2018-12-31],
      dtype='object', name='date', length=631)

In [208]:
c = df_trump_approval[df_trump_approval['timestamp'].isin(a.index)]
d = df_trump_approval[df_trump_approval['timestamp'].isin(b.index)]
stats.ttest_ind(c['approve_estimate'],d['approve_estimate'], equal_var = False)



Ttest_indResult(statistic=-0.43092443703885364, pvalue=0.6672753457013607)

In [209]:
c = df_stocks[df_stocks.index.isin(a.index)]
d = df_stocks[df_stocks.index.isin(b.index)]
stats.ttest_ind(c['5. adjusted close'],d['5. adjusted close'], equal_var = False)

Ttest_indResult(statistic=-7.750745549493501, pvalue=4.6528643888092445e-12)

In [210]:
print(((c['5. adjusted close']- c['1. open'])/c['1. open']).mean())
print(((d['5. adjusted close']- d['1. open'])/d['1. open']).mean())

-0.032923070059652094
-0.02492375691714304
